![Img](https://app.theheadstarter.com/static/hs-logo-opengraph.png)

# Headstarter Codebase RAG Project

![Screenshot 2024-11-25 at 7 12 58 PM](https://github.com/user-attachments/assets/0bd67cf0-43d5-46d2-879c-a752cae4c8e3)

# Install Necessary Libraries

In [2]:
! pip install pygithub langchain langchain-community openai tiktoken pinecone-client langchain_pinecone sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.9/375.9 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 61.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 51.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 54.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 58.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.5/409.5 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 46.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 72.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.6 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Success

In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from langchain_pinecone import PineconeVectorStore
from langchain.embeddings import OpenAIEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings
from google.colab import userdata
from pinecone import Pinecone
import os
import tempfile
from github import Github, Repository
from git import Repo
from openai import OpenAI
from pathlib import Path
from langchain.schema import Document
from pinecone import Pinecone

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


# Clone a GitHub Repo locally

In [ ]:
def clone_repository(repo_url):
    """Clones a GitHub repository to a temporary directory.

    Args:
        repo_url: The URL of the GitHub repository.

    Returns:
        The path to the cloned repository.
    """
    repo_name = repo_url.split("/")[-1]  # Extract repository name from URL
    repo_path = f"/content/{repo_name}" # Set the path where the repo will be saved in our colab environment
    Repo.clone_from(repo_url, str(repo_path))
    return str(repo_path)

In [ ]:
path = clone_repository("https://github.com/tinaf2/AI-Coding-Agent")

In [ ]:
print(path)

/content/AI-Coding-Agent


In [ ]:
SUPPORTED_EXTENSIONS = {'.py', '.js', '.tsx', '.jsx', '.ipynb', '.java',
                         '.cpp', '.ts', '.go', '.rs', '.vue', '.swift', '.c', '.h'}

IGNORED_DIRS = {'node_modules', 'venv', 'env', 'dist', 'build', '.git',
                '__pycache__', '.next', '.vscode', 'vendor'}

In [ ]:
# Remove the existing Node.js version
!sudo apt-get purge -y nodejs
!sudo apt-get autoremove -y


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following packages will be REMOVED:
  nodejs*
0 upgraded, 0 newly installed, 1 to remove and 57 not upgraded.
After this operation, 197 MB disk space will be freed.
(Reading database ... 129008 files and directories currently installed.)
Removing nodejs (20.18.1-1nodesource1) ...
dpkg: warning: while removing nodejs, directory '/usr/lib/node_modules' not empty so not removed
Processing triggers for man-db (2.10.2-1) ...
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
0 upgraded, 0 newly installed, 0 to remove and 57 not upgraded.


In [ ]:
# Install Node.js 20 LTS (Long Term Support)
!curl -fsSL https://deb.nodesource.com/setup_20.x | bash -
!sudo apt-get install -y nodejs

# Verify the Node.js version
!node --version


2024-11-30 02:44:12 - Installing pre-requisites
Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 https://deb.nodesource.com/node_20.x nodistro InRelease
Hit:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Fetched 384 kB in 2s (242 kB/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.s

In [ ]:
# Install TypeScript globally
!npm install -g typescript

# Verify TypeScript installation
!tsc --version


⠙⠹⠸⠼⠴⠦⠧⠇
changed 1 package in 1s
⠇Version 5.7.2


In [ ]:
import subprocess
import json

def extract_functions_from_code(code, file_name):
    functions = []

    # Write the JavaScript code to extract function names and content using TypeScript's AST
    js_code = f"""
const ts = require('typescript');

function extractFunctions(code) {{
    try {{
        const sourceFile = ts.createSourceFile(
            'temp.ts',  // Temporary file name
            code,
            ts.ScriptTarget.Latest,
            true  // Set true for better parsing accuracy
        );

        const functions = [];

        function visit(node) {{
            if (ts.isFunctionDeclaration(node) || ts.isArrowFunction(node)) {{
                const functionName = node.name ? node.name.text : 'anonymous';
                const functionContent = code.slice(node.getStart(), node.getEnd());  // Extract full function content
                functions.push({{name: functionName, content: functionContent}});
            }} else if (ts.isMethodDeclaration(node)) {{
                // Detect methods and ensure they are inside classes
                if (node.parent && ts.isClassDeclaration(node.parent)) {{
                    const functionName = node.name ? node.name.text : 'anonymous';
                    const functionContent = code.slice(node.getStart(), node.getEnd());  // Extract method content
                    functions.push({{name: functionName, content: functionContent}});
                }}
            }}
            ts.forEachChild(node, visit);
        }}

        ts.forEachChild(sourceFile, visit);

        return functions;
    }} catch (e) {{
        console.error("Error in AST processing:", e.message);
        return [];
    }}
}}

const code = `{code}`;  // Pass the code as a string

const functionNames = extractFunctions(code);

// Output the function names and content as a JSON array
console.log(JSON.stringify(functionNames));
"""

    # Run the JavaScript code with Node.js and capture the output
    result = subprocess.run(['node', '-e', js_code], capture_output=True, text=True)

    # Clean the raw output to extract only the JSON part
    raw_output = result.stdout.strip()

    if not raw_output:
        return []

    # Split the output to isolate the JSON part (after the first part of the string)
    json_part = raw_output.split("\n")[-1].strip()

    if json_part == "[]":
        return []

    # Replace single quotes with double quotes to ensure it's valid JSON
    json_part = json_part.replace("'", '"')

    try:
        function_data = json.loads(json_part)
    except json.JSONDecodeError as e:
        print("Error parsing JSON:", e)
        function_data = []

    # Create a list of function documents
    documents = []
    for function in function_data:
        function_name = function["name"]
        function_content = function["content"]

        # Convert escaped newline characters to actual newlines for the content
        function_content = function_content.replace("\\n", "\n")  # Replace '\\n' with actual newlines

        # Create a document for each function
        document = Document(
            page_content=function_content,  # Store full function content with proper newlines
            metadata={"function_name": function_name, "file_name": file_name}
        )
        documents.append(document)

    return documents


In [ ]:
def get_file_content(file_path, repo_path):
    """
    Get content of a single file.

    Args:
        file_path (str): Path to the file

    Returns:
        Optional[Dict[str, str]]: Dictionary with file name and content
    """
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()

        # Get relative path from repo root
        rel_path = os.path.relpath(file_path, repo_path)

        return {
            "name": rel_path,
            "content": content
        }
    except Exception as e:
        print(f"Error processing file {file_path}: {str(e)}")
        return None


def get_main_files_content(repo_path: str):
    """
    Get content of supported code files from the local repository.

    Args:
        repo_path: Path to the local repository

    Returns:
        List of dictionaries containing file names and contents
    """
    files_content = []

    try:
        for root, _, files in os.walk(repo_path):
            # Skip if current directory is in ignored directories
            if any(ignored_dir in root for ignored_dir in IGNORED_DIRS):
                continue

            # Process each file in current directory
            for file in files:
                file_path = os.path.join(root, file)
                if os.path.splitext(file)[1] in SUPPORTED_EXTENSIONS:
                    file_content = get_file_content(file_path, repo_path)
                    if file_content:
                        files_content.append(file_content)

    except Exception as e:
        print(f"Error reading repository: {str(e)}")

    return files_content

In [ ]:
file_content = get_main_files_content(path)

In [ ]:
file_content

[{'name': 'src/prompts.ts',
  'content': 'import { encode, encodeChat } from "gpt-tokenizer";\nimport type { ChatCompletionMessageParam } from "groq-sdk/resources/chat/completions";\nimport type { PRFile } from "./constants";\nimport {\n  rawPatchStrategy,\n  smarterContextPatchStrategy,\n} from "./context/review";\nimport { GROQ_MODEL, type GroqChatModel } from "./llms/groq";\n\nconst ModelsToTokenLimits: Record<GroqChatModel, number> = {\n  "mixtral-8x7b-32768": 32768,\n  "gemma-7b-it": 32768,\n  "llama3-70b-8192": 8192,\n  "llama3-8b-8192": 8192,\n};\n\nexport const REVIEW_DIFF_PROMPT = `You are PR-Reviewer, a language model designed to review git pull requests.\nYour task is to provide constructive and concise feedback for the PR, and also provide meaningful code suggestions.\n\nExample PR Diff input:\n\'\n## src/file1.py\n\n@@ -12,5 +12,5 @@ def func1():\ncode line that already existed in the file...\ncode line that already existed in the file....\n-code line that was removed in t

# Embeddings

In [ ]:
def get_huggingface_embeddings(text, model_name="sentence-transformers/all-mpnet-base-v2"):
    model = SentenceTransformer(model_name)
    return model.encode(text)

In [ ]:
text = "I am a programmer"

embeddings = get_huggingface_embeddings(text)

In [ ]:
embeddings

array([ 1.81737654e-02, -3.02657508e-03, -4.77465875e-02,  1.86379403e-02,
        3.14537995e-02,  1.87255293e-02, -1.52534274e-02, -6.77293688e-02,
       -1.26903653e-02,  1.28427576e-02,  5.80701306e-02,  4.00234833e-02,
        3.27073298e-02,  7.12998435e-02,  5.56373373e-02,  1.68628506e-02,
        6.97603747e-02, -5.02619930e-02,  6.13140827e-03, -1.46559235e-02,
       -4.51957993e-03,  4.82934639e-02, -2.53051296e-02, -1.97862904e-03,
       -4.36902530e-02, -2.41507161e-02,  1.29505759e-02, -3.78611824e-03,
       -2.05718316e-02,  1.09819308e-01,  3.07672890e-03, -2.80443169e-02,
       -1.55807249e-02, -1.24789868e-02,  1.75239131e-06, -2.93756695e-03,
       -1.43048428e-02,  4.88386713e-02, -6.21114224e-02,  2.95061413e-02,
       -1.40470508e-02,  2.20708270e-02,  1.13067888e-02,  4.70893271e-02,
        7.58305984e-03, -8.30314530e-05,  6.67821169e-02, -1.21320095e-02,
        4.39386303e-03,  2.47453637e-02,  1.02529004e-02, -6.54432410e-03,
       -5.53147821e-03, -

# Setting up Pinecone
**1. Create an account on [Pinecone.io](https://app.pinecone.io/)**

**2. Create a new index called "codebase-rag" and set the dimensions to 768. Leave the rest of the settings as they are.**

![Screenshot 2024-11-24 at 10 58 50 PM](https://github.com/user-attachments/assets/f5fda046-4087-432a-a8c2-86e061005238)



**3. Create an API Key for Pinecone**

![Screenshot 2024-11-24 at 10 44 37 PM](https://github.com/user-attachments/assets/e7feacc6-2bd1-472a-82e5-659f65624a88)


**4. Store your Pinecone API Key within Google Colab's secrets section, and then enable access to it (see the blue checkmark)**

![Screenshot 2024-11-24 at 10 45 25 PM](https://github.com/user-attachments/assets/eaf73083-0b5f-4d17-9e0c-eab84f91b0bc)



In [ ]:
# Set the PINECONE_API_KEY as an environment variable
pinecone_api_key = userdata.get("PINECONE_API_KEY")
os.environ['PINECONE_API_KEY'] = pinecone_api_key

# Initialize Pinecone
pc = Pinecone(api_key=userdata.get("PINECONE_API_KEY"),)

# Connect to your Pinecone index
pinecone_index = pc.Index("codebase-rag")

In [ ]:
vectorstore = PineconeVectorStore(index_name="codebase-rag", embedding=HuggingFaceEmbeddings())

<ipython-input-250-5982ffb8f713>:1: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  vectorstore = PineconeVectorStore(index_name="codebase-rag", embedding=HuggingFaceEmbeddings())


In [ ]:
# Code for embedding individual functions (narrower context, best for more granular questions):

# Initialize an empty list to store documents
documents = []

# Extract functions from each file and create documents for each function
for file in file_content:
    # Extract the function documents from the file's content
    functions = extract_functions_from_code(file['content'], file['name'])

    # Append each extracted function's Document into the documents list
    documents.extend(functions)  # Since extract_functions_from_code already returns Document objects

# Ensure we have documents before proceeding with Pinecone
if documents:
    # Create Pinecone VectorStore from the documents
    vectorstore = PineconeVectorStore.from_documents(
        documents=documents,
        embedding=HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2"),  # Explicitly specify model name
        index_name="codebase-rag",
        namespace="https://github.com/tinaf2/AI-Coding-Agent"
    )
else:
    print("No functions extracted, so no documents were created.")

# Code for embedding entire files (creates wider context, best for for answering broader questions):
"""
documents = []

for file in file_content:
    doc = Document(
        page_content=f"{file['name']}\n{file['content']}",
        metadata={"source": file['name']}
    )

    documents.append(doc)


vectorstore = PineconeVectorStore.from_documents(
    documents=documents,
    embedding=HuggingFaceEmbeddings(),
    index_name="codebase-rag",
    namespace="https://github.com/tinaf2/AI-Coding-Agent"
)
"""

# Perform RAG

1. Get your Groq API Key [here](https://console.groq.com/keys)

2. Paste your Groq API Key into your Google Colab secrets, and make sure to enable permissions for it

![Screenshot 2024-11-25 at 12 00 16 AM](https://github.com/user-attachments/assets/e5525d29-bca6-4dbd-892b-cc770a6b281d)


In [ ]:
client = OpenAI(
    base_url="https://api.groq.com/openai/v1",
    api_key=userdata.get("GROQ_API_KEY")
)

In [ ]:
# query = "How are JavaScript files parsed?"
# query = "How does the function findEnclosingContext() work?"
query = "What does the dryRun() method do?"

In [ ]:
query_embedding = get_huggingface_embeddings(query)

query_embedding

array([-2.69781929e-02, -8.93092379e-02, -2.32615769e-02,  4.50351201e-02,
       -3.12609002e-02,  3.39495838e-02,  4.75559942e-03,  2.27411129e-02,
        4.42190990e-02, -5.33255003e-03,  4.18482721e-03, -2.01086570e-02,
        6.75915629e-02, -5.30218473e-03, -1.26705961e-02,  1.64977852e-02,
        1.10236313e-02, -3.50672333e-03,  4.99543622e-02,  1.12223281e-02,
       -3.62162255e-02, -6.38258038e-03, -1.82863057e-03, -3.91646922e-02,
        2.95535363e-02, -1.07172690e-02, -3.47760655e-02,  6.73763081e-02,
       -3.04532815e-02, -2.81910375e-02, -9.72602144e-03,  2.09326185e-02,
        2.40743477e-02, -3.20834443e-02,  1.49147468e-06, -9.59868170e-03,
        2.32664701e-02,  1.96962040e-02, -1.00538507e-02,  9.64815840e-02,
        2.60188729e-02,  6.34844005e-02,  4.54158969e-02,  4.94143181e-02,
       -2.63134600e-03,  4.39222306e-02,  1.51665993e-02, -3.78171578e-02,
        3.45307915e-03, -1.44038233e-03, -1.52776539e-02,  6.72779651e-03,
       -3.98582742e-02,  

In [ ]:
# Feel free to change the "top_k" parameter to be a higher or lower number
top_matches = pinecone_index.query(vector=query_embedding.tolist(), top_k=5, include_metadata=True, namespace="https://github.com/tinaf2/AI-Coding-Agent")

In [ ]:
top_matches

{'matches': [{'id': 'e68ffddb-1ee9-4e69-a9bc-57b02c2797ff',
              'metadata': {'file_name': 'src/context/language/javascript-parser.ts',
                           'function_name': 'dryRun',
                           'text': 'async dryRun(file: string): Promise<{ '
                                   'valid: boolean; error: string }> {\n'
                                   '    try {\n'
                                   '      const ast = parser.parse(file, {\n'
                                   '        sourceType: "module",\n'
                                   '        plugins: ["jsx", "typescript"], // '
                                   'To allow JSX and TypeScript\n'
                                   '      });\n'
                                   '      return {\n'
                                   '        valid: true,\n'
                                   '        error: "",\n'
                                   '      };\n'
                                   '  

In [ ]:
contexts = [item['metadata']['text'] for item in top_matches['matches']]

In [ ]:
contexts

['async dryRun(file: string): Promise<{ valid: boolean; error: string }> {\n    try {\n      const ast = parser.parse(file, {\n        sourceType: "module",\n        plugins: ["jsx", "typescript"], // To allow JSX and TypeScript\n      });\n      return {\n        valid: true,\n        error: "",\n      };\n    } catch (exc) {\n      return {\n        valid: false,\n        error: exc,\n      };\n    }\n  }',
 'async dryRun(file: string): Promise<{ valid: boolean; error: string }> {\n    try {\n      const ast = parser.parse(file, {\n        sourceType: "module",\n        plugins: ["jsx", "typescript"], // To allow JSX and TypeScript\n      });\n      return {\n        valid: true,\n        error: "",\n      };\n    } catch (exc) {\n      return {\n        valid: false,\n        error: exc,\n      };\n    }\n  }',
 'async dryRun(file: string): Promise<{ valid: boolean; error: string }> {\n    try {\n      const ast = parser.parse(file, {\n        sourceType: "module",\n        plugins:

In [ ]:
augmented_query = "<CONTEXT>\n" + "\n\n-------\n\n".join(contexts[ : 10]) + "\n-------\n</CONTEXT>\n\n\n\nMY QUESTION:\n" + query

In [ ]:
print(augmented_query)

<CONTEXT>
async dryRun(file: string): Promise<{ valid: boolean; error: string }> {
    try {
      const ast = parser.parse(file, {
        sourceType: "module",
        plugins: ["jsx", "typescript"], // To allow JSX and TypeScript
      });
      return {
        valid: true,
        error: "",
      };
    } catch (exc) {
      return {
        valid: false,
        error: exc,
      };
    }
  }

-------

async dryRun(file: string): Promise<{ valid: boolean; error: string }> {
    try {
      const ast = parser.parse(file, {
        sourceType: "module",
        plugins: ["jsx", "typescript"], // To allow JSX and TypeScript
      });
      return {
        valid: true,
        error: "",
      };
    } catch (exc) {
      return {
        valid: false,
        error: exc,
      };
    }
  }

-------

async dryRun(file: string): Promise<{ valid: boolean; error: string }> {
    try {
      const ast = parser.parse(file, {
        sourceType: "module",
        plugins: ["jsx", "typesc

In [ ]:
system_prompt = f"""You are an expert Software Engineer, with over 20 years of experience in TypeScript and Python.

Answer any questions I have about the codebase, based on the code provided. Always consider all of the context provided when forming a response."""


llm_response = client.chat.completions.create(
    model="llama-3.1-8b-instant",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": augmented_query}
    ]
)

response = llm_response.choices[0].message.content

In [ ]:
response

'The `dryRun()` method appears to be a linting or validation function that checks the syntax of a given JavaScript or TypeScript file without executing it. \n\nIn other words, it takes a file path as input, parses the code using the `parser`, and checks if the code is syntactically valid without actually executing it. This is often referred to as a "dry run."\n\nHere\'s a step-by-step breakdown:\n\n1. It attempts to parse the code from the given file using the `parser.parse()` method.\n2. If the code is parsed successfully, the method returns an object with a `valid` property set to `true` and an `error` property set to an empty string (`""`).\n3. If there\'s an error during parsing (e.g., syntax error), the method catches the exception, sets `valid` to `false`, and includes the error message as the value for the `error` property.\n\nThis method likely serves as a way to quickly check if a file has any syntax errors or other issues without actually running the code, making it a useful 

# Putting it all together

In [ ]:
def perform_rag(query):
    query_embedding = get_huggingface_embeddings(query)

    top_matches = pinecone_index.query(vector=query_embedding.tolist(), top_k=5, include_metadata=True, namespace="https://github.com/tinaf2/AI-Coding-Agent")

    # Get the list of retrieved texts
    contexts = [item['metadata']['text'] for item in top_matches['matches']]

    augmented_query = "<CONTEXT>\n" + "\n\n-------\n\n".join(contexts[ : 10]) + "\n-------\n</CONTEXT>\n\n\n\nMY QUESTION:\n" + query

    # Modify the prompt below as need to improve the response quality
    system_prompt = f"""You are a Senior Software Engineer, specializing in TypeScript.

    Answer any questions I have about the codebase, based on the code provided. Always consider all of the context provided when forming a response.
    """

    llm_response = client.chat.completions.create(
        model="llama-3.1-70b-versatile",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": augmented_query}
        ]
    )

    return llm_response.choices[0].message.content

In [ ]:
response = perform_rag("How are javascript files parsed?")

print(response)

Based on the context provided, it appears that JavaScript files are parsed using a parser that is not explicitly defined in the given code snippets. However, the parser used seems to be compatible with the Babylon parser, a JavaScript parser used by Babel.

The parser parses the JavaScript files with options to support JSX and TypeScript syntax. Here's the relevant line of code:

```typescript
const ast = parser.parse(file, {
  sourceType: "module",
  plugins: ["jsx", "typescript"], // To allow JSX and TypeScript
});
```

In this code snippet:

- `sourceType` is set to `"module"`, which means the parser is expecting a JavaScript module (as opposed to a JavaScript script).
- The `plugins` option is set to include `"jsx"` and `"typescript"`. This enables support for JSX syntax (a syntax extension for JavaScript) and TypeScript syntax.

By parsing the JavaScript files with these options, the code is able to produce an Abstract Syntax Tree (AST) representation of the code that includes sup

In [1]:
# Code for deleted Pinecone data

"""
# Initialize Pinecone client and the index you want to delete data from
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
pinecone_index = pc.Index("codebase-rag")

# Delete all vectors in the existing index
pinecone_index.delete(delete_all=True)

"""


ModuleNotFoundError: No module named 'pinecone'